In [18]:
import pandas as pd
import torch
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import *

data = pd.read_csv('./criteo_sample.txt')

# 稀疏特征与稠密特征
sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I' + str(i) for i in range(1, 14)]

# 稀疏特征填充-1
# 稠密特征填充0
data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0, )
target = ['label']
data

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,0.0,3,260.0,0.0,17668.0,0.0,0.0,33.0,0.0,...,e5ba7672,87c6f83c,-1,-1,0429f84b,-1,3a171ecb,c0d61a5c,-1,-1
1,0,0.0,-1,19.0,35.0,30251.0,247.0,1.0,35.0,160.0,...,d4bb7bd8,6fc84bfb,-1,-1,5155d8a3,-1,be7c41b4,ded4aac9,-1,-1
2,0,0.0,0,2.0,12.0,2013.0,164.0,6.0,35.0,523.0,...,e5ba7672,675c9258,-1,-1,2e01979f,-1,bcdee96c,6d5d1302,-1,-1
3,0,0.0,13,1.0,4.0,16836.0,200.0,5.0,4.0,29.0,...,e5ba7672,52e44668,-1,-1,e587c466,-1,32c7478e,3b183c5c,-1,-1
4,0,0.0,0,104.0,27.0,1990.0,142.0,4.0,32.0,37.0,...,e5ba7672,25c88e42,21ddcdc9,b1252a9d,0e8585d2,-1,32c7478e,0d4a6d1a,001f3601,92c878de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0.0,0,113.0,3.0,3036.0,575.0,2.0,3.0,214.0,...,07c540c4,9880032b,21ddcdc9,5840adea,34cc61bb,c9d4222a,32c7478e,e5ed7da2,ea9a246c,984e0db0
196,1,0.0,1,1.0,1.0,1607.0,12.0,1.0,12.0,15.0,...,1e88c74f,3972b4ed,-1,-1,d1aa4512,-1,32c7478e,9257f75f,-1,-1
197,1,1.0,0,6.0,3.0,0.0,0.0,19.0,3.0,3.0,...,3486227d,5aed7436,54591762,a458ea53,4a2c3526,-1,32c7478e,1793a828,e8b83407,1a02cbe1
198,0,0.0,22,6.0,22.0,203.0,153.0,80.0,18.0,508.0,...,3486227d,13145934,55dd3565,5840adea,bf647035,-1,32c7478e,1481ceb4,e8b83407,988b0775


In [19]:
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
data[sparse_features]

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,4,96,146,1,4,163,1,1,72,...,8,66,0,0,3,0,1,96,0,0
1,11,1,98,98,1,6,179,0,1,89,...,7,52,0,0,47,0,7,112,0,0
2,0,18,39,52,3,4,140,2,1,93,...,8,49,0,0,25,0,6,53,0,0
3,0,45,7,117,1,0,164,1,0,20,...,8,37,0,0,156,0,0,32,0,0
4,0,11,59,77,1,5,18,1,1,45,...,8,14,5,3,9,0,0,5,1,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0,66,107,1,4,10,15,1,11,...,0,74,5,1,30,5,0,118,17,48
196,21,64,44,65,1,6,4,2,1,9,...,1,25,0,0,138,0,0,68,0,0
197,0,5,153,143,1,0,141,1,1,118,...,4,40,17,2,41,0,0,12,16,11
198,0,84,84,41,5,4,27,1,1,96,...,4,7,18,1,123,0,0,10,16,49


In [20]:
mms = MinMaxScaler(feature_range=(0,1))
data[dense_features] = mms.fit_transform(data[dense_features])
data[dense_features]

,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13
0,0.000000,0.001332,0.092362,0.000000,0.034825,0.000000,0.000000,0.673469,0.000000,0.000000,0.00000,0.0,0.000000
1,0.000000,0.000000,0.006750,0.402299,0.059628,0.117284,0.003322,0.714286,0.154739,0.000000,0.03125,0.0,0.343137
2,0.000000,0.000333,0.000710,0.137931,0.003968,0.077873,0.019934,0.714286,0.505803,0.000000,0.09375,0.0,0.176471
3,0.000000,0.004664,0.000355,0.045977,0.033185,0.094967,0.016611,0.081633,0.028046,0.000000,0.06250,0.0,0.039216
4,0.000000,0.000333,0.036945,0.310345,0.003922,0.067426,0.013289,0.653061,0.035783,0.000000,0.03125,0.0,0.264706
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.000000,0.000333,0.040142,0.034483,0.005984,0.273029,0.006645,0.061224,0.206963,0.000000,0.03125,0.0,0.029412
196,0.000000,0.000666,0.000355,0.011494,0.003168,0.005698,0.003322,0.244898,0.014507,0.000000,0.03125,0.0,0.117647
197,0.027027,0.000333,0.002131,0.034483,0.000000,0.000000,0.063123,0.061224,0.002901,0.333333,0.28125,0.0,0.000000
198,0.000000,0.007662,0.002131,0.252874,0.000400,0.072650,0.265781,0.367347,0.491296,0.000000,0.34375,0.0,0.215686


In [30]:
sparse_features_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(),embedding_dim=4) for i,feat in enumerate(sparse_features)]
dense_features_columns = [DenseFeat(feat, 1,) for feat in dense_features]
sparse_features_columns

[SparseFeat(name='C1', vocabulary_size=27, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C1', group_name='default_group'),
 SparseFeat(name='C2', vocabulary_size=92, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C2', group_name='default_group'),
 SparseFeat(name='C3', vocabulary_size=172, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C3', group_name='default_group'),
 SparseFeat(name='C4', vocabulary_size=157, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C4', group_name='default_group'),
 SparseFeat(name='C5', vocabulary_size=12, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C5', group_name='default_group'),
 SparseFeat(name='C6', vocabulary_size=7, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C6', group_name='default_group'),
 SparseFeat(name='C7', vocabulary_size=183, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C7', group_name='default_group'),
 SparseFeat

In [32]:
feature_names = get_feature_names(dense_features_columns + sparse_features_columns)
feature_names

['I1',
 'I2',
 'I3',
 'I4',
 'I5',
 'I6',
 'I7',
 'I8',
 'I9',
 'I10',
 'I11',
 'I12',
 'I13',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21',
 'C22',
 'C23',
 'C24',
 'C25',
 'C26']

In [33]:
train, test = train_test_split(data, test_size=0.2)
train

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
193,0,0.000000,0.000999,0.008171,0.229885,0.000292,0.000000,0.000000,0.408163,0.019342,...,5,40,1,2,18,0,0,39,18,84
188,1,0.000000,0.000666,0.002487,0.022989,0.005736,0.000950,1.000000,0.061224,0.052224,...,8,11,42,2,119,0,0,57,12,81
67,0,0.027027,0.000666,0.001776,0.206897,0.000936,0.029915,0.049834,0.081633,0.776596,...,8,113,5,1,11,0,0,94,7,30
184,1,0.054054,0.002332,0.000000,0.000000,0.000591,0.011871,0.006645,0.510204,0.065764,...,8,98,0,0,52,0,0,9,0,0
21,0,0.000000,0.001666,0.000000,0.000000,0.003099,0.000000,0.000000,0.346939,0.053191,...,5,92,32,3,67,0,7,37,2,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,0,0.000000,0.018654,0.003552,0.137931,0.000589,0.000000,0.000000,0.469388,0.025145,...,5,84,0,0,24,3,7,65,0,0
26,0,0.000000,0.000666,0.000710,0.183908,0.028392,0.037512,0.006645,0.326531,0.099613,...,0,93,0,0,121,5,0,2,0,0
16,0,0.000000,0.008328,0.001066,0.022989,0.020095,0.000000,0.000000,0.653061,0.053191,...,8,53,0,0,76,0,1,87,0,0
129,0,0.054054,0.000333,0.002131,0.022989,0.000138,0.004748,0.823920,0.020408,1.000000,...,8,34,5,1,43,0,0,80,18,48


In [39]:
train_model_input = {name:train[name] for name in feature_names}
test_model_input = {name:test[name] for name in feature_names}

device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

model = afm.AFM(sparse_features_columns+dense_features_columns, sparse_features_columns,task='binary',device=device)
model.compile("adam", "binary_crossentropy",
              metrics=['binary_crossentropy'], )

history = model.fit(train_model_input,train[target].values,batch_size=256,epochs=10,verbose=2,validation_split=0.2)
pred_ans = model.predict(test_model_input, batch_size=256)

cuda ready...
cuda:0
Train on 128 samples, validate on 32 samples, 1 steps per epoch
Epoch 1/10
0s - loss:  0.6932 - binary_crossentropy:  0.6932 - val_binary_crossentropy:  0.6905
Epoch 2/10
0s - loss:  0.6830 - binary_crossentropy:  0.6830 - val_binary_crossentropy:  0.6881
Epoch 3/10
0s - loss:  0.6730 - binary_crossentropy:  0.6730 - val_binary_crossentropy:  0.6858
Epoch 4/10
0s - loss:  0.6632 - binary_crossentropy:  0.6632 - val_binary_crossentropy:  0.6836
Epoch 5/10
0s - loss:  0.6535 - binary_crossentropy:  0.6535 - val_binary_crossentropy:  0.6814
Epoch 6/10
0s - loss:  0.6439 - binary_crossentropy:  0.6439 - val_binary_crossentropy:  0.6793
Epoch 7/10
0s - loss:  0.6345 - binary_crossentropy:  0.6345 - val_binary_crossentropy:  0.6773
Epoch 8/10
0s - loss:  0.6252 - binary_crossentropy:  0.6252 - val_binary_crossentropy:  0.6753
Epoch 9/10
0s - loss:  0.6161 - binary_crossentropy:  0.6161 - val_binary_crossentropy:  0.6734
Epoch 10/10
0s - loss:  0.6072 - binary_crossentrop